# Storage Backends and Mount Points

**Duration:** 20 minutes  
**Level:** Beginner

Learn how to configure and work with different storage backends.

## What You'll Learn

- Different storage backend types
- How to configure multiple backends
- Mount point naming and best practices
- Backend capabilities
- Configuration from files (YAML/JSON)

## Storage Backends Available

- **Local**: Local filesystem
- **Memory**: In-memory storage (testing)
- **S3**: Amazon S3
- **GCS**: Google Cloud Storage
- **Azure**: Azure Blob Storage
- **HTTP**: Read-only HTTP/HTTPS
- **Base64**: Embedded base64 data

Let's explore! 🗂️

In [ ]:
from genro_storage import StorageManager
import tempfile
import os

storage = StorageManager()
print("✓ Storage manager ready!")

## 1. Memory Backend

Perfect for testing, temporary data, and examples:

In [ ]:
storage.configure([
    {'name': 'temp', 'type': 'memory'}
])

# Create a temporary file
temp_file = storage.node('temp:test.txt')
temp_file.write_text('This data exists only in memory')

print(f"✓ Memory storage configured")
print(f"File exists: {temp_file.exists}")
print(f"Content: {temp_file.read_text()}")

# Check capabilities
caps = temp_file.capabilities
print(f"\nCapabilities:")
print(f"  Read: {caps.read}")
print(f"  Write: {caps.write}")
print(f"  Temporary: {caps.temporary}")
print(f"  Versioning: {caps.versioning}")

## 2. Local Filesystem Backend

Access files on your computer:

In [ ]:
# Create a temporary directory for this example
local_dir = tempfile.mkdtemp()

storage.configure([
    {'name': 'local', 'type': 'local', 'path': local_dir}
], replace=False)

# Create a real file on disk
local_file = storage.node('local:document.txt')
local_file.write_text('This file is on disk!')

print(f"✓ Local storage configured")
print(f"Base path: {local_dir}")
print(f"File path: {local_file.fullpath}")
print(f"Actually exists on disk: {os.path.exists(local_file.fullpath)}")

# Cleanup will happen at the end

## 3. Multiple Mount Points

You can configure multiple backends at once:

In [ ]:
# Create multiple temp directories
home_dir = tempfile.mkdtemp()
work_dir = tempfile.mkdtemp()
uploads_dir = tempfile.mkdtemp()

# Configure multiple backends
storage.configure([
    {'name': 'home', 'type': 'local', 'path': home_dir},
    {'name': 'work', 'type': 'local', 'path': work_dir},
    {'name': 'uploads', 'type': 'local', 'path': uploads_dir},
    {'name': 'cache', 'type': 'memory'}
], replace=False)

print("✓ Multiple storage backends configured")
print(f"Available mounts: {', '.join(storage.get_mount_names())}")

# Use them
storage.node('home:personal.txt').write_text('Personal file')
storage.node('work:project.txt').write_text('Work file')
storage.node('uploads:photo.jpg').write_text('Photo data...')
storage.node('cache:session.tmp').write_text('Temporary cache')

print("\n✓ Files created in different locations")

## 4. Copying Between Backends

Seamlessly copy files between different storage types:

In [ ]:
# Create a file in memory
mem_report = storage.node('cache:report.txt')
mem_report.write_text('Q4 Sales Report\n\nRevenue: $1M\nGrowth: 15%')

# Copy to home
home_report = storage.node('home:reports/q4.txt')
home_report.parent.mkdir(parents=True, exist_ok=True)
mem_report.copy(home_report)

# Copy to work
work_report = storage.node('work:archive/2024/q4.txt')
work_report.parent.mkdir(parents=True, exist_ok=True)
mem_report.copy(work_report)

print("✓ Report copied to multiple locations")
print(f"  Memory: {mem_report.fullpath}")
print(f"  Home: {home_report.fullpath}")
print(f"  Work: {work_report.fullpath}")
print(f"\nAll exist: {mem_report.exists and home_report.exists and work_report.exists}")

## 5. HTTP Backend (Read-Only)

Access files from the web:

In [ ]:
# Configure HTTP backend
storage.configure([
    {'name': 'web', 'type': 'http', 'base_url': 'https://raw.githubusercontent.com'}
], replace=False)

# Access a file from GitHub
readme = storage.node('web:genropy/genro-storage/main/README.md')

print(f"✓ HTTP storage configured")
print(f"File exists: {readme.exists}")

# Read first 200 characters
content = readme.read_text()
print(f"\nFirst 200 chars of README:")
print(content[:200] + "...")

# Check capabilities
print(f"\nHTTP capabilities:")
print(f"  Read: {readme.capabilities.read}")
print(f"  Write: {readme.capabilities.write}")
print(f"  Read-only: {readme.capabilities.readonly}")

## 6. Base64 Backend

Work with embedded base64 data:

In [ ]:
import base64

# Configure base64 backend
storage.configure([
    {'name': 'b64', 'type': 'base64'}
], replace=False)

# Create some data
data = "Hello, Base64!"
encoded = base64.b64encode(data.encode()).decode()

# Create node with base64 path
b64_node = storage.node(f'b64:{encoded}')

print(f"✓ Base64 storage configured")
print(f"Original data: {data}")
print(f"Encoded: {encoded}")
print(f"Decoded from node: {b64_node.read_text()}")

# Useful for embedding small files in URLs or configs
print(f"\nUse case: data:image/png;base64,{encoded}")

## 7. S3 Configuration (AWS)

Configure Amazon S3 storage (requires `pip install genro-storage[s3]`):

In [ ]:
# Example S3 configuration (won't actually run without credentials)

s3_config = {
    'name': 's3',
    'type': 's3',
    'bucket': 'my-app-bucket',
    # Optional:
    'region': 'us-east-1',
    'prefix': 'uploads/',  # All paths relative to this prefix
    'aws_access_key_id': 'YOUR_KEY',
    'aws_secret_access_key': 'YOUR_SECRET'
}

print("S3 Configuration example:")
print(f"  Bucket: {s3_config['bucket']}")
print(f"  Region: {s3_config.get('region', 'default')}")
print(f"  Prefix: {s3_config.get('prefix', 'root')}")

# To actually use:
# storage.configure([s3_config])
# file = storage.node('s3:document.pdf')

print("\nNote: S3 requires proper AWS credentials to work")

## 8. Configuration from YAML

Load configuration from a YAML file:

In [ ]:
import yaml

# Create a YAML config file
config_yaml = """
mounts:
  - name: data
    type: local
    path: /var/data
    
  - name: cache
    type: memory
    
  - name: backup
    type: s3
    bucket: my-backups
    region: eu-west-1
"""

# Parse and show
config = yaml.safe_load(config_yaml)
print("YAML Configuration:")
print(yaml.dump(config, default_flow_style=False))

# To use:
# storage.configure('config.yaml')
# or
# storage.configure(config['mounts'])

## 9. Configuration from JSON

Or use JSON:

In [ ]:
import json

# JSON configuration
config_json = """
{
  "mounts": [
    {
      "name": "uploads",
      "type": "local",
      "path": "/var/uploads"
    },
    {
      "name": "cdn",
      "type": "s3",
      "bucket": "my-cdn",
      "public": true
    }
  ]
}
"""

config = json.loads(config_json)
print("JSON Configuration:")
print(json.dumps(config, indent=2))

# To use:
# storage.configure('config.json')
# or
# storage.configure(config['mounts'])

## 10. Checking Configured Mounts

You can inspect what's configured:

In [ ]:
# List all mount names
mounts = storage.get_mount_names()
print(f"Configured mounts ({len(mounts)}):")
for mount in sorted(mounts):
    print(f"  - {mount}")

# Check if specific mount exists
print(f"\nHas 'home' mount: {storage.has_mount('home')}")
print(f"Has 's3' mount: {storage.has_mount('s3')}")

## 11. Backend Capabilities

Each backend reports its capabilities:

In [ ]:
# Compare capabilities across backends
def show_capabilities(mount_name):
    node = storage.node(f'{mount_name}:test')
    caps = node.capabilities
    
    print(f"\n{mount_name}:")
    print(f"  Read: {caps.read}")
    print(f"  Write: {caps.write}")
    print(f"  Delete: {caps.delete}")
    print(f"  Versioning: {caps.versioning}")
    print(f"  Metadata: {caps.metadata}")
    print(f"  URLs: {caps.presigned_urls}")
    print(f"  Read-only: {caps.readonly}")
    print(f"  Temporary: {caps.temporary}")

print("Backend Capabilities Comparison:")
show_capabilities('cache')
show_capabilities('home')
show_capabilities('web')

## 12. Dynamic Configuration

You can add or replace mounts at runtime:

In [ ]:
print(f"Before: {storage.get_mount_names()}")

# Add a new mount
new_temp = tempfile.mkdtemp()
storage.configure([
    {'name': 'new_mount', 'type': 'local', 'path': new_temp}
], replace=False)

print(f"After adding: {storage.get_mount_names()}")

# Replace an existing mount
storage.configure([
    {'name': 'cache', 'type': 'memory'}  # Reconfigure cache
], replace=True)  # This only replaces 'cache'

print(f"After replacing: {storage.get_mount_names()}")

## 13. Try It Yourself! 🎯

**Exercise 1:** Configure three mounts: 'dev', 'staging', 'prod' (all memory for now)

In [ ]:
# Your code here


**Exercise 2:** Create a config file, copy it between dev/staging/prod:

In [ ]:
# Your code here


**Exercise 3:** Create a function that shows all files across all mounts:

In [ ]:
def list_all_files(storage_manager):
    """List all files across all mounts"""
    # Your code here
    pass

# list_all_files(storage)

## 14. Cleanup

Clean up temporary directories:

In [ ]:
import shutil

# Clean up all temp directories we created
temp_dirs = [local_dir, home_dir, work_dir, uploads_dir, new_temp]
for d in temp_dirs:
    if os.path.exists(d):
        shutil.rmtree(d)

print("✓ Cleanup complete")

## Summary

You've learned:

- ✓ Different storage backend types
- ✓ How to configure multiple mounts
- ✓ Copying between different backends
- ✓ Backend capabilities and limitations
- ✓ Configuration from YAML/JSON
- ✓ Dynamic mount management

## Key Concepts

- **Mount point**: Named storage backend (e.g., `home:`, `s3:`)
- **Backend**: Underlying storage implementation
- **Capabilities**: What a backend can/cannot do
- **Cross-backend copy**: Seamless data transfer

## What's Next?

Continue to:

- **[03_file_operations.ipynb](03_file_operations.ipynb)** - Master all file operations
- **[04_virtual_nodes.ipynb](04_virtual_nodes.ipynb)** - Learn about virtual nodes

Happy configuring! ⚙️